In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import datetime as dt
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import random
from pandas_profiling import ProfileReport
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Lars
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import LassoLars
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_percentage_error
import seaborn as sns

In [3]:
energy = pd.read_csv(r"C:\Users\Admin\OneDrive\Desktop\KAG_energydata_complete.csv")
energy.shape

(19735, 29)

In [4]:
#converting date into datetime
energy['date'] = energy['date'].astype('datetime64[ns]')
energy.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [5]:
#making the date, time, hour and month into a diff column and thus converting them into date time format
energy['Date'] = pd.to_datetime(energy['date']).dt.date
energy['Time'] = pd.to_datetime(energy['date']).dt.time
energy['hour'] = energy['date'].dt.hour
energy['month'] = energy['date'].dt.month

In [6]:
energy.drop(['Date', 'Time'], axis = 1 , inplace = True)

In [7]:
target = 'Appliances'

In [8]:
train_df, test_df = train_test_split(energy, test_size=0.25, random_state=40)

In [9]:
train_df = train_df.reset_index(drop=True)

In [10]:
train_df

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,hour,month
0,2016-03-17 23:40:00,70,20,22.600,34.700000,19.500000,37.863333,21.500000,35.345000,22.890000,...,2.233333,764.200000,87.666667,1.333333,61.000000,0.400000,43.195492,43.195492,23,3
1,2016-05-25 04:40:00,50,0,23.700,40.290000,21.000000,42.900000,25.390000,40.000000,23.600000,...,6.266667,757.233333,93.666667,1.000000,25.333333,5.300000,46.369677,46.369677,4,5
2,2016-03-26 19:40:00,100,20,22.200,45.800000,20.463333,44.930000,23.000000,38.560000,20.390000,...,13.366667,750.133333,66.333333,6.333333,40.000000,7.133333,25.722968,25.722968,19,3
3,2016-04-28 01:10:00,40,0,21.290,35.790000,18.230000,37.863333,21.290000,34.790000,20.533333,...,0.316667,755.216667,94.333333,2.000000,20.166667,-0.450000,3.791656,3.791656,1,4
4,2016-02-01 05:40:00,50,10,21.290,47.400000,20.290000,47.290000,21.666667,46.760000,18.790000,...,10.766667,758.666667,92.666667,6.666667,59.000000,9.633333,8.706206,8.706206,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14796,2016-03-31 19:00:00,100,0,21.760,41.933333,20.000000,43.163333,22.000000,38.466667,20.600000,...,8.100000,754.800000,82.000000,2.000000,40.000000,5.200000,36.275509,36.275509,19,3
14797,2016-05-02 06:30:00,80,0,21.200,36.433333,18.000000,40.200000,22.100000,35.090000,20.260000,...,1.750000,764.700000,92.500000,1.000000,27.500000,0.650000,37.879262,37.879262,6,5
14798,2016-04-21 09:50:00,50,0,21.600,37.466667,22.200000,35.700000,22.700000,35.700000,22.390000,...,9.966667,762.616667,67.500000,4.000000,27.166667,4.100000,22.496162,22.496162,9,4
14799,2016-04-21 18:50:00,120,0,23.175,37.722500,22.856667,35.560000,23.290000,36.590000,22.890000,...,17.366667,759.633333,47.500000,4.000000,38.166667,5.916667,32.362623,32.362623,18,4


In [11]:
X_train = train_df.drop(target, axis=1)
y_train = train_df[target]
X_test = test_df.drop(target, axis=1)
y_test = test_df[target]

In [12]:
X_train = train_df.drop('date', axis=1)
X_test = test_df.drop('date', axis=1)

In [13]:
# define the pipeline
pipeline = [ExtraTreesRegressor(),
            RandomForestRegressor(),
            LGBMRegressor(),
            KNeighborsRegressor(),
            GradientBoostingRegressor(),
            Lars(),
            Ridge(),
            LinearRegression(),
            BayesianRidge(),
            Lasso(),
            ElasticNet(),
            OrthogonalMatchingPursuit(),
            HuberRegressor(),
            LassoLars(),
            DummyRegressor(),
            PassiveAggressiveRegressor(),
            DecisionTreeRegressor(),
            AdaBoostRegressor(),
            XGBRegressor() 
            ]

for i in pipeline:
  print(i)
  # fit the model
  i.fit(X_train, y_train)
  # evaluate the model
  pred = i.predict(X_test)
  # evaluate predictions
  rmse = np.sqrt(MSE(y_test, pred))
  print("RMSE : % f" %(rmse))
  mae = mean_absolute_error(y_test,pred)
  print("MAE : % f" %(mae))
  print("R2 : % f" %(i.score(X_test,y_test)))
  mape = mean_absolute_percentage_error(y_test, pred)
  print("MAPE : % f" %(mape))
  print()

ExtraTreesRegressor()
RMSE :  0.317475
MAE :  0.025294
R2 :  0.999990
MAPE :  0.000129

RandomForestRegressor()
RMSE :  1.459058
MAE :  0.055188
R2 :  0.999793
MAPE :  0.000085

LGBMRegressor()
RMSE :  3.830200
MAE :  0.183610
R2 :  0.998570
MAPE :  0.001534

KNeighborsRegressor()
RMSE :  4.667545
MAE :  3.122416
R2 :  0.997877
MAPE :  0.049937

GradientBoostingRegressor()
RMSE :  0.385545
MAE :  0.060243
R2 :  0.999986
MAPE :  0.000412

Lars()
RMSE :  0.000000
MAE :  0.000000
R2 :  1.000000
MAPE :  0.000000

Ridge()
RMSE :  0.000001
MAE :  0.000000
R2 :  1.000000
MAPE :  0.000000

LinearRegression()


C:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lars())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(


RMSE :  0.000000
MAE :  0.000000
R2 :  1.000000
MAPE :  0.000000

BayesianRidge()
RMSE :  0.000000
MAE :  0.000000
R2 :  1.000000
MAPE :  0.000000

Lasso()
RMSE :  0.009563
MAE :  0.005636
R2 :  1.000000
MAPE :  0.000072

ElasticNet()
RMSE :  0.009563
MAE :  0.005636
R2 :  1.000000
MAPE :  0.000072

OrthogonalMatchingPursuit()
RMSE :  0.000000
MAE :  0.000000
R2 :  1.000000
MAPE :  0.000000

HuberRegressor()


C:\Anaconda\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), OrthogonalMatchingPursuit())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_omp.py:758: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  coef_, self.n_iter_ = orthogonal_mp_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_huber.py

RMSE :  0.290215
MAE :  0.231470
R2 :  0.999992
MAPE :  0.003679

LassoLars()
RMSE :  101.305613
MAE :  59.709863
R2 : -0.000031
MAPE :  0.764259

DummyRegressor()
RMSE :  101.305613
MAE :  59.709863
R2 : -0.000031
MAPE :  0.764259

PassiveAggressiveRegressor()
RMSE :  0.052084
MAE :  0.042820
R2 :  1.000000
MAPE :  0.000730

DecisionTreeRegressor()
RMSE :  0.667747
MAE :  0.032428
R2 :  0.999957
MAPE :  0.000041

AdaBoostRegressor()
RMSE :  9.813761
MAE :  7.396646
R2 :  0.990615
MAPE :  0.153319

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_de